# Analyzing borrowers’ risk of defaulting


## Introduction

This report verifies whether a customer will default on a loan is depending on their martial status and number of children they have. This report will serve as a reference for building a credit score for a potential customer, as a credit score is used to evulate the ability of a potential borrower to repay their loan.  

### Goal:

This report will test four hypotheses listed below:
1. There is an association between having kids and repaying a loan on time. 
2. There is an association between marital status and repaying a loan on time. 
3. There is an association between income level and repaying a loan on time. 
4. Different loan purposes affect on-time loan repayment.

### Stages:

This project will consit of the following stages:
1. Data overview
2. Data preprocessing
3. Working with missing values
4. Categorizing data
5. Hypotheses testing



## Stage 1. Data overview. 





Open the data file and have a look at the general information. 



In [1]:
# Loading all the libraries
import pandas as pd

# Load the data

score = pd.read_csv('/datasets/credit_scoring_eng.csv')


In [2]:
# Let's see how many rows and columns our dataset has

print(score.shape)

(21525, 12)


In [3]:
# let's print the first 20 rows

print(score.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  SECONDARY EDUCATION             1   
8          2   -6929.865299         35    BACHELOR'S DEGREE             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

In [4]:
# Get info on data
score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


This dataset contains 12 columns and 21525 rows. 

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


we can see there are several issues with this dataset:
1. Potential missing values in columns 'days_employed' and 'total_income'. 
2. Some entries in 'education' appeared to be capitalized, although they are referring to the same value.
3. Description of 'purpose' is unclear, different terms were used to described the same purpose.



<div class="alert alert-block alert-success">
<b>Reviewer's comment v1</b>  Correct

In [5]:
# Let's look in the filtered table at the the first column with missing data

print(score['days_employed'].isna().value_counts(dropna=False))

False    19351
True      2174
Name: days_employed, dtype: int64


In [6]:
# To see the total missing value in the dataframe

print(score.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Although the total number of missing values appeared to be the same in 'days_empolyed' and 'total_income', we cannot simply assume that they are related. Further conditional filtering is required. 

In [7]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

print(len(score[(score['days_employed'].isna()) & (score['dob_years'] >= 65)]))

print(len(score[(score['days_employed'].isna()) & (score['total_income'].isna())]))

print(len(score[(score['income_type'] != 'retiree') & (score['days_employed'].isna())]))

print(len(score[(score['family_status_id'] == 0) & (score['days_employed'].isna())]))

print(len(score[(score['children'] == 0) & (score['days_employed'].isna())]))

print(len(score[(score['education_id'] !=0) & (score['days_employed'].isna())]))

print(len(score[(score['debt'] != 1) & (score['days_employed'].isna())]))

81
2174
1761
1237
1439
1630
2004


Based on the result of filtering with conditions, missing values in column 'days_employed' and 'total_income' are the same size. However, that only means that whenever there is a missing value in 'days_empolyed', a missing value is also observed in 'total_income'. The missing values in both columns could potentially be explained by other client characterstics. In which, multiple logical conditions were applied. It appeared that 2004 of the missing values correlate with cilents that do not have any debt. 1761 correlates with cilents that are not retiree, 1630 with clients that do not have a bachelor degree, follow by 1237 with cilents are legally married, and 1439 with cilents that have no children.   

From the filtering it appeared that the missing values might be random. However, further investigations are required. 



In [8]:
# percentage of the missing values compared to the whole dataset.
print(score.isna().sum()*100 / len(score))


children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64


**Intermediate conclusion**



Filtering with conditions showed that missing values in column'days_employed' and 'total_income' appeared to be the same size. This leads to the suspicion of that the missing values in 'days_employed' might be due to that they have no income or vice-versa. Therefore they are not able to record any response. If this theory is true, then we should have a missing 'days_employed' value for every respondent who also does not have an income (missing value 'NaN'). As what the next step will be testing, the return for unique value should be 'nan'. However, I would also want to the test the relationship between educational level and missing value in both "total_income' and 'days_employed'. Since a large portion of missing value appeared to be associated with educational level.  

The dependence that missing value might have on the value of other indicators with the columns with identified specific client characteristic will also be tested by checking the distribution. Thus, checking the randomness of missing data. Whether the distribution of the rows with missing values has similar appearance as the whole dataset.  

Additionally, the calculated percentage of the missing values compared to the whole dataset is ~10.1%. Whether we should fill the missing values remains to be determined, this will depend whether they are randomly missing.   


In [9]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
print(score[score['days_employed'].isna()]['total_income'].unique())


[nan]


Confirming that when there is a missing value in 'days_employed', a missing value is also observed in 'toatl_income' and vice versa. Next to test the association between 'education_id' and 'days_employed'/'total_income'. 

In [10]:
# Investigating cilents who do not have data on identified characterstics and the column with the missing values. 

print(score[score['days_employed'].isna()]['education_id'].value_counts())
print(score[score['total_income'].isna()]['education_id'].value_counts())

1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64
1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64


In [11]:
# Checking distribution
print(score[score['days_employed'].isna()]['education_id'].value_counts())
print(score[score['total_income'].isna()]['education_id'].value_counts())


1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64
1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64




It appeared that educational_id = 1 has the made up most of the missing values, therefore, clients with secondary education is more often to have a missing value in 'days_employed' and 'total_income'. 


**Possible reasons for missing values in data**

The possible reason for the missing value could be due to not having much education as others, although that does not account for all of the missing values. Therefore the missing values might be random. 

The randomness of the missing data will be tested in the following steps. 



In [12]:
# Checking the distribution in the whole dataset
print(score[score['days_employed'].isna()]['education_id'].value_counts())
print(score[score['days_employed'].isna()]['children'].value_counts())
print(score[score['days_employed'].isna()]['income_type'].value_counts())
print(score[score['days_employed'].isna()]['family_status'].value_counts())
print(score[score['days_employed'].isna()]['debt'].value_counts())
print(score[score['days_employed'].isna()]['gender'].value_counts())

score.describe()


1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64
married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64
0    2004
1     170
Name: debt, dtype: int64
F    1484
M     690
Name: gender, dtype: int64


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Intermediate conclusion**



Upon the examination of all other categorical columns of the dataset and by calling score.describe(), the distribution of the missing values among those columns appeared to be similar to the filtered table with 'education_id'. The appearance of missing data does not seem to have a pattern. In which a correlation between the missing data and other values in the rows where they appear has not been detected. Suggesting that they are most likely random. 

Additional checking, with other indicators have been done (see results for all categorical columns). For clarity, they will be listed again for more explanation. 



In [13]:
# Checking for other patterns - debt

print(score[score['days_employed'].isna()]['debt'].value_counts())


0    2004
1     170
Name: debt, dtype: int64


In [14]:
# Checking for other patterns - family_status

print(score[score['days_employed'].isna()]['family_status'].value_counts())



married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64


**Intermediate conclusion**

Again, no pattern between potential indicator and the missing values are observed. Not all missing values were associated with clients with 'debt' or 'married'. Therefore, suggesting missing values are accidental. 



**Conclusions**


To conclude, no patterns between missing values observed in 'days_employed' and 'total_income' and indicators such as 'education_id', 'children', 'family_status', 'debt', 'income_type' were found. The missing values were like to be random and only covers ~10% of the data. Therefore, the most appropriate way to address those values will be to replace them with a value, since they are quantitative. However, whether the datset contains significant outliers should first be determined. Method usage like mean() and median() will depend on the presence of outliers. 

In the next steps, data will be transformed first by checking duplicates and dropping them. Issues identified by the first glance of the dataset will also be addressed such as different registers, incorrect artifacts. And finally to replace missing values with either mean() or median() using fillna(). 


## Stage 2. Data preprocessing

Going through each column and check for potential problems. 


**Starting with 'education' column.**

In [15]:
# Let's see all values in education column to check if and what spellings will need to be fixed

print(score['education'].unique())


["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


Some registers/entries in 'education' appeared to be capitalized(first letter or all-capitalized), although they are referring to the same value

In [16]:
# Fix the registers using the long way

def replace_wrong_registers (wrong, correct):
    
    #looping over the list
    
    for wrong in wrong:
        
        #replacing the wrong registers with the correct ones
        score['education'] = score['education'].replace(wrong, correct)

#List of all of the wrong registers
duplicates1 = ['Secondary Education', 'SECONDARY EDUCATION']
duplicates2 = ["BACHELOR'S DEGREE", "Bachelor's Degree"]
duplicates3 = ['Primary Education', 'PRIMARY EDUCATION']
duplicates4 = ['GRADUATE DEGREE','Graduate Degree']
duplicates5 = ['SOME COLLEGE','Some College']


#correct registers
correct1 = 'secondary education'
correct2 = "bachelor's degree"
correct3 = 'primary education'
correct4 = 'graduate degree'
correct5 = 'some college'


#replacement
replace_wrong_registers(duplicates1, correct1)
replace_wrong_registers(duplicates2, correct2)
replace_wrong_registers(duplicates3, correct3)
replace_wrong_registers(duplicates4, correct4)
replace_wrong_registers(duplicates5, correct5)



In [17]:
# Checking all the values in the column to make sure we fixed them

print(sorted(score['education'].unique()))

["bachelor's degree", 'graduate degree', 'primary education', 'secondary education', 'some college']


In [18]:
# Fix the registers using the short way - I realized that it is just capitalization

score['education'] = score['education'].str.lower()

In [19]:
# To check the fix has worked

print(sorted(score['education'].unique()))

["bachelor's degree", 'graduate degree', 'primary education', 'secondary education', 'some college']


**Checking the 'children' column.**

In [20]:
# Let's see the distribution of values in the `children` column
print(score['children'].unique())

[ 1  0  3  2 -1  4 20  5]


In [21]:
# Checking the percentage of the errorous data

print(score['children'].value_counts() / len(score['children']) *100)

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64



There appeared to have a negative number of children, which likely is an entry error. Moreover, the value 20 also appear to be an entry error. 

By checking the occurrence of the problematic data, it revealed that they only contribute to a very small percentage of the data, 0.35% for 20, and 0.21% for -1. Since the contribution is so slim, I can either drop the problematic rows or replace -1 with 1, and remove the 0 in 20. I will replace it for now. 


In [22]:
# [fix the data based on your decision]

score['children'] = score['children'].replace(-1, 1)
score['children'] = score['children'].replace(20, 2)

Thank you!! I'll keep that in mind!

In [23]:
# Checking the `children` column again to make sure it's all fixed

print(score['children'].unique())


[1 0 3 2 4 5]


**Checking the 'days_employed' column.**

In [24]:
# Find problematic data in `days_employed` & percentage

print(score['days_employed'].unique())

print(len(score[score['days_employed'] < 0]) / len(score['days_employed']) *100)

[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
73.89547038327527


From the few lines that showed up, a problem immediately jumped out, as that most of the data contain negative numbers. It is illogical to have negative number of employment days. 

The reason for negative number appearing in the dataset is perhaps due to either a technical error in formating the table; it could be hitting a button that makes a number of rows negative in 'days_employed', or using the wrong date to subtract (start date - end date instead of the reversal) or human error in recording the data.

The correct data should have no negative numbers, since negative numbers account for ~74% of the data, we cannot simply drop the rows. therefore the next step will be to absolute all of the values in 'days_employed'.

Of course, there are still missing values, but I will deal with the negative numbers first.

In [25]:
# Address the problematic values - negative number

score['days_employed'] = score['days_employed'].abs()

In [26]:
# Check the result - make sure it's fixed

print(score['days_employed'].unique())

[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]


**Checking the 'dob_years' column.** 

In [27]:
# Check the `dob_years` for suspicious values and count the percentage

print(score['dob_years'].unique())

print(score['dob_years'].value_counts() / len(score['dob_years']) *100)

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
35    2.866434
40    2.829268
41    2.819977
34    2.801394
38    2.778165
42    2.773519
33    2.699187
39    2.662021
31    2.601626
36    2.578397
44    2.541231
29    2.531940
30    2.508711
48    2.499419
37    2.494774
50    2.387921
43    2.383275
32    2.369338
49    2.360046
28    2.336818
45    2.308943
27    2.290360
56    2.262485
52    2.248548
47    2.229965
54    2.225319
46    2.206736
58    2.141696
57    2.137050
53    2.132404
51    2.081301
59    2.062718
55    2.058072
26    1.895470
60    1.751452
25    1.658537
61    1.649245
62    1.635308
63    1.249710
64    1.231127
24    1.226481
23    1.180023
65    0.901278
66    0.850174
22    0.850174
67    0.775842
21    0.515679
0     0.469222
68    0.459930
69    0.394890
70    0.301974
71    0.269454
20    0.236934
72    0.153310
19    0.0650

The problem here is that "0" cannot be someone's age when they are trying to apply for a loan. 
However, the percentage of this problematic data is only 0.46%. I can either drop the problematic rows or attend to replace it with the most logical number, assuming is 30. I will replace it with 30 for now, as such a small percentage of data will be unlikely to impact the dataset and subsequent analyses.

In [28]:
# Address the issues in the `dob_years` column, if they exist

score['dob_years'] = score['dob_years'].replace(0, 30)

In [29]:
# Check the result - make sure it's fixed

print(sorted(score['dob_years'].unique()))

[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]




**Checking the 'family_status' column.**

In [30]:
# Let's see the values for the column

print(score['family_status'].unique())

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


For clarity, 'widow/widower' can be simpified to 'widowed', which 'civil partnership', 'widowed' and 'divorced' can both be 'unmarried', so I will change 'unmarried' to 'never married'. 

In [31]:
# Address the problematic values in `family_status`, if they exist

score['family_status'] = score['family_status'].replace('widow / widower', 'widowed')

score['family_status'] = score['family_status'].replace('unmarried', 'never married')

In [32]:
# Check the result - make sure it's fixed

print(score['family_status'].unique())

['married' 'civil partnership' 'widowed' 'divorced' 'never married']



**Checking the 'gender' column.**

In [33]:
# Let's see the values in the column

print(score['gender'].unique())

print(score['gender'].value_counts())

['F' 'M' 'XNA']
F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Although there is the curious 'XNA', since it only accounts for a very small percentage of the dataset (~0.005%), it can be either dropped or ignored. I will drop the XNA since there is only 1 respondent with this answer, it will unlikely to impact the analyses. 

In [34]:
# Address the problematic values - dropping XNA

score = score.loc[score['gender'] != 'XNA']

In [35]:
# Check the result - make sure it's fixed

print(score['gender'].unique())


['F' 'M']


Down to two values!



**Checking the 'income_type' column.**

In [36]:
# Let's see the values in the column

print(score['income_type'].unique())

print(score['income_type'].value_counts())

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']
employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


For clarity, 'entrepreneur' can be categorized under 'business' without the need of creating a new category. I am unsure of what paternity/maternity leave means in this case as whether they still has an income from employer, and what does student means, so I will leave those two be. 

In [37]:
# Address the problematic values

score['income_type'] = score['income_type'].replace('entrepreneur', 'business')


In [38]:
# Check the result - make sure it's fixed

print(score['income_type'].unique())

['employee' 'retiree' 'business' 'civil servant' 'unemployed' 'student'
 'paternity / maternity leave']



**Checking for duplicates.**

In [39]:
# Checking duplicates

print(score[score.duplicated(keep = False)].sort_values(by=['dob_years', 'education','family_status']))

print(score.duplicated(keep = False).sum())


       children  days_employed  dob_years            education  education_id  \
8853          1            NaN         23  secondary education             1   
20297         1            NaN         23  secondary education             1   
15892         0            NaN         23  secondary education             1   
19321         0            NaN         23  secondary education             1   
3452          0            NaN         29    bachelor's degree             0   
...         ...            ...        ...                  ...           ...   
5865          0            NaN         66  secondary education             1   
9528          0            NaN         66  secondary education             1   
6537          0            NaN         71  secondary education             1   
7938          0            NaN         71  secondary education             1   
9604          0            NaN         71  secondary education             1   

           family_status  family_status

There appeared to be 137 true duplicates, but since we are keeping the first entry of the duplicates, the number of duplicates drop will be different. Will proceed to remove the obvious duplicates, since this dataset does not have a column of IDs designated for each respondent, it would be hard to check whether there are other types of duplicates. 

In [40]:
# Address the duplicates,reset index and dropping the old index.

score = score.drop_duplicates().reset_index(drop=True)

In [41]:
# Last check whether we have any duplicates

print(score.duplicated().sum())


0


In [42]:
# Check the size of the dataset that you now have after your first manipulations with it

print(score.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21453 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In conclusion, the 'education' column has been fixed, where strings are now identical by making them lowercase with str.lower(). 

The problematic entries -1 and 20 found in 'children' has been replaced with logical values. 

'days_employed' consisted of ~73% of negative values, which was problematic. An absoulte has been taken for the whole column, changing negative numbers to positive. 

The 0 found in 'dob_years' column has been replaced with a logical value of 30.

Some registers under 'family_status' have been modified to make it more clear. 

One entry of 'XNA' found in 'gender' column has been removed.

'entrepreneur' was merged with 'business' for better representaion in 'income_type' column. While leaving 'student', 'paternity/maternity leave' untouched as unsure of where they qualify. 

Finally, 71 obvious duplicates were found using .duplicated() and removed with drop_duplicates().

Newly processed dataset now has 21453 rows, as compared to 21525 to begin with. So a total of 72 rows were removed, counting for roughly 0.3% of the dataset. 


## Stage 3. Working with missing values

### Restoring missing values in `total_income`



Missing values were identified in 'total_income' and 'days_employed'. I will first check the distribution of the data (done), to determine whether there are significant outliers present. If there are outliers then I will use median() to fill missing values and use mean() when there are no outliers. 

I will create a category for age based on the respondent's age. I used the increment of 10 for clarity and better representation. 


In [44]:
# A function that calculates the age category

def age_category(age):
    if age < 20:
        return '<20'
    if 20 <= age <= 29:
        return '20-29'
    if 30 <= age <= 39:
        return '30-39'
    if 40 <= age <= 49:
        return '40-49'
    if 50 <= age <= 59:
        return '50-59'
    if 60 <= age <= 69:
        return '60-69'
    if age >= 70:
        return '70+'
    

In [45]:
# Test if the function works
print(age_category(55))
print(age_category(19))
print(age_category(38))

50-59
<20
30-39


In [46]:
# Creating new column based on function

score['age_category'] = score['dob_years'].apply(age_category)


In [47]:
# Checking how values in the new column

print(score.head(20))
print(score['age_category'].value_counts())

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42    bachelor's degree             0   
1          1    4024.803754         36  secondary education             1   
2          0    5623.422610         33  secondary education             1   
3          3    4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0     926.185831         27    bachelor's degree             0   
6          0    2879.202052         43    bachelor's degree             0   
7          0     152.779569         50  secondary education             1   
8          2    6929.865299         35    bachelor's degree             0   
9          0    2188.756445         41  secondary education             1   
10         2    4171.483647         36    bachelor's degree             0   
11         0     792.701887         40  secondary education             1   


Income depends on multiple factors: experience (in this case that is related to days_employed and/or age), educational level and the type of jobs. 

Will proceed to check how different indicators affect the mean and median of total income. 

Creating a table with no missing values to to check the mean and median of income based on different indicators. 

In [48]:
# Create a table without missing values and print a few of its rows to make sure it looks fine

score_nomissing = score.dropna()
print(score_nomissing.head(15))


    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42    bachelor's degree             0   
1          1    4024.803754         36  secondary education             1   
2          0    5623.422610         33  secondary education             1   
3          3    4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0     926.185831         27    bachelor's degree             0   
6          0    2879.202052         43    bachelor's degree             0   
7          0     152.779569         50  secondary education             1   
8          2    6929.865299         35    bachelor's degree             0   
9          0    2188.756445         41  secondary education             1   
10         2    4171.483647         36    bachelor's degree             0   
11         0     792.701887         40  secondary education             1   

In [49]:
score_nomissing.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19350.000000,19350.000000,19350.000000,19350.000000,19350.000000,19350.000000,19350.000000
mean,0.479638,66918.065141,43.397416,0.819018,0.972248,0.081189,26787.266688
std,0.753906,139033.698578,12.256880,0.550053,1.420633,0.273132,16475.822926
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,926.990457,33.000000,1.000000,0.000000,0.000000,16486.515250
50%,0.000000,2194.218768,42.000000,1.000000,0.000000,0.000000,23201.873500
75%,1.000000,5538.423086,53.000000,1.000000,1.000000,0.000000,32547.910750
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [50]:
# Look at the mean values for income based on your identified factors

#factors =  'gender', 'debt', 'age_category','children', 'family status', 'education'

print(score_nomissing.groupby('age_category')['total_income'].mean())
print(score_nomissing.groupby('gender')['total_income'].mean())
print(score_nomissing.groupby('debt')['total_income'].mean())
print(score_nomissing.groupby('children')['total_income'].mean())
print(score_nomissing.groupby('education')['total_income'].mean())
print(score_nomissing.groupby('family_status')['total_income'].mean())


age_category
20-29    25570.172966
30-39    28260.357840
40-49    28551.375635
50-59    25811.700327
60-69    23242.812818
70+      20125.658331
<20      16993.942462
Name: total_income, dtype: float64
gender
F    24655.604757
M    30907.144369
Name: total_income, dtype: float64
debt
0    26848.336178
1    26096.143537
Name: total_income, dtype: float64
children
0    26421.916832
1    27368.627863
2    27478.854282
3    29322.623993
4    27289.829647
5    27268.847250
Name: total_income, dtype: float64
education
bachelor's degree      33142.802434
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.503037
some college           29040.132990
Name: total_income, dtype: float64
family_status
civil partnership    26692.840381
divorced             27189.354550
married              27041.784689
never married        26934.069805
widowed              22984.208556
Name: total_income, dtype: float64


In [51]:
# Look at the median values for income based on your identified factors

print(score_nomissing.groupby('age_category')['total_income'].median())
print(score_nomissing.groupby('gender')['total_income'].median())
print(score_nomissing.groupby('debt')['total_income'].median())
print(score_nomissing.groupby('children')['total_income'].median())
print(score_nomissing.groupby('education')['total_income'].median())
print(score_nomissing.groupby('family_status')['total_income'].median())


age_category
20-29    22798.6650
30-39    24664.8260
40-49    24764.2290
50-59    22203.0745
60-69    19817.4400
70+      18751.3240
<20      14934.9010
Name: total_income, dtype: float64
gender
F    21464.845
M    26834.295
Name: total_income, dtype: float64
debt
0    23225.474
1    22928.480
Name: total_income, dtype: float64
children
0    23027.3350
1    23660.5630
2    23136.1155
3    25155.4480
4    24981.6340
5    29816.2255
Name: total_income, dtype: float64
education
bachelor's degree      28054.5310
graduate degree        25161.5835
primary education      18741.9760
secondary education    21836.5830
some college           25608.7945
Name: total_income, dtype: float64
family_status
civil partnership    23185.477
divorced             23515.096
married              23389.540
never married        23149.028
widowed              20514.190
Name: total_income, dtype: float64


I looked at the following factors to check on their impact on total_income:  

'gender', 'debt', 'age_category','children', 'family status', 'education level'

There is a large difference in income between male and female. The different is subtle in debt. It appeared that the number of children also has an impact on the income but that could be due to the total number of respondent in each category since the number of people that has 3+ more kids are significantly less than others. Similar can be said for educcatioanl level. The difference in family status is also subtle. Therefore I feel that age group is a better representation since the number of people fit into the category is similarly distributed and it covers a wild range of age group. 

The presence of outlier in 'total_income' suggested using the median() to fill the missing values instead of mean(). Since median is more resilient to the presence of outlier. 

In [52]:
#  Write a function that we will use for filling in missing values

def replacement(income):
    income_med = score_nomissing['total_income'].median()
    
    if pd.isnull(income) == True:
        return income_med
    else:
        return income
    

    

In [55]:
# Check if it works

print(replacement(20))


20


In [56]:
# Apply it to every row

score['total_income'] = score['total_income'].apply(replacement)

In [57]:
# Check if we got any errors

print(score.head(13))


    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42    bachelor's degree             0   
1          1    4024.803754         36  secondary education             1   
2          0    5623.422610         33  secondary education             1   
3          3    4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0     926.185831         27    bachelor's degree             0   
6          0    2879.202052         43    bachelor's degree             0   
7          0     152.779569         50  secondary education             1   
8          2    6929.865299         35    bachelor's degree             0   
9          0    2188.756445         41  secondary education             1   
10         2    4171.483647         36    bachelor's degree             0   
11         0     792.701887         40  secondary education             1   

In [58]:
# Checking the number of entries in the columns

print(score.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB
None


Missing values are filled with the median in total_income column.

###  Restoring values in `days_employed`

I looked at the following factors to check on their impact on days_employed:  

'gender', 'debt', 'age_category','children', 'family status', 'education level'

With reasons similar to 'total_income'. 

In [59]:
# Distribution of `days_employed` medians based on your identified parameters

print(score_nomissing.groupby('age_category')['days_employed'].median())
print(score_nomissing.groupby('gender')['days_employed'].median())
print(score_nomissing.groupby('debt')['days_employed'].median())
print(score_nomissing.groupby('children')['days_employed'].median())
print(score_nomissing.groupby('education')['days_employed'].median())
print(score_nomissing.groupby('family_status')['days_employed'].median())

age_category
20-29      1004.573780
30-39      1606.283237
40-49      2111.489906
50-59      4796.767897
60-69    354935.619093
70+      361336.993449
<20         724.492610
Name: days_employed, dtype: float64
gender
F    2539.856894
M    1662.370103
Name: days_employed, dtype: float64
debt
0    2267.408539
1    1498.192818
Name: days_employed, dtype: float64
children
0    2625.852176
1    1669.174020
2    1678.132084
3    1765.066044
4    1905.879025
5    1231.571486
Name: days_employed, dtype: float64
education
bachelor's degree      1895.747795
graduate degree        5660.057032
primary education      3043.933615
secondary education    2392.483500
some college           1206.479128
Name: days_employed, dtype: float64
family_status
civil partnership      1943.995523
divorced               2401.954568
married                2304.964439
never married          1462.009287
widowed              337017.713307
Name: days_employed, dtype: float64


In [60]:
# Distribution of `days_employed` means based on your identified parameters

print(score_nomissing.groupby('age_category')['days_employed'].mean())
print(score_nomissing.groupby('gender')['days_employed'].mean())
print(score_nomissing.groupby('debt')['days_employed'].mean())
print(score_nomissing.groupby('children')['days_employed'].mean())
print(score_nomissing.groupby('education')['days_employed'].mean())
print(score_nomissing.groupby('family_status')['days_employed'].mean())


age_category
20-29      2088.960274
30-39      5298.848078
40-49     12383.580460
50-59    132907.545543
60-69    283926.481689
70+      320819.151927
<20         633.678086
Name: days_employed, dtype: float64
gender
F    82397.770891
M    37000.331325
Name: days_employed, dtype: float64
debt
0    68946.730621
1    43959.666946
Name: days_employed, dtype: float64
children
0    92526.171104
1    23454.811112
2     6686.606176
3     9338.376355
4    13863.043444
5     1432.348601
Name: days_employed, dtype: float64
education
bachelor's degree       42375.409174
graduate degree        121323.630206
primary education      130340.426349
secondary education     76413.822372
some college            20683.780431
Name: days_employed, dtype: float64
family_status
civil partnership     58411.268077
divorced              68816.335483
married               63312.782890
never married         47072.691647
widowed              205636.887848
Name: days_employed, dtype: float64


There is a large difference in days_employed between male and female (for unknown reason). The different is also large in debt. It appeared that the number of children also has an impact on the income but that could be due to the total number of respondent in each category since the number of people that has 3+ more kids are significantly less than others. Similar can be said for educcatioanllevel. the indicator family status returned some very interesting results that I don't think it will be good representation. Therefore I feel that age group is again, a better representation since the days_employed are often associated with the age of respondents. 

The presence of extreme outlier in 'days_employed' suggested using the median() to fill the missing values instead of mean(). Since median is more resilient to the presence of outlier.

In [61]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter

def fill(days):
    days_mean = score_nomissing['days_employed'].mean()
    
    if pd.isnull(days) == True:
        return days_mean
    else:
        return days
    
# Single line function:

#score['days_employed'] = score['days_employed'].fillna(score_nomissing['days_employed'].median())

In [62]:
# Check that the function works

print(fill(4349))

4349


In [63]:
# Apply function to the income_type

score['days_employed'] = score['days_employed'].apply(fill)

In [64]:
# Replacing missing values

print(score.head(13))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42    bachelor's degree             0   
1          1    4024.803754         36  secondary education             1   
2          0    5623.422610         33  secondary education             1   
3          3    4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0     926.185831         27    bachelor's degree             0   
6          0    2879.202052         43    bachelor's degree             0   
7          0     152.779569         50  secondary education             1   
8          2    6929.865299         35    bachelor's degree             0   
9          0    2188.756445         41  secondary education             1   
10         2    4171.483647         36    bachelor's degree             0   
11         0     792.701887         40  secondary education             1   

In [65]:
# Check the entries in all columns - make sure we fixed all missing values

print(score.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB
None


The total number of rows matched with others - all missing values have been filled.

## Stage 4. Categorization of data


I chose three categorical column and one numerical to categorize. 

Recall the hypothesis:

1. There is an association between having kids and repaying a loan on time. ('children', 'debt') 
2. There is an association between marital status and repaying a loan on time. ('family_status','debt')
3. There is an association between income level and repaying a loan on time. ('total_income','debt')
4. Different loan purposes affect on-time loan repayment ('purpose', 'debt')

'children','family_status', and 'purpose' are categorical columns, while 'total_income' is numerical. Currently, there are multiple entries under each column, to get a better idea, they should be categorized into a clear, more representative way. 'debt' only has two registers, so I will leave it be. Detailed explanation will be found under each categorization. 



In [66]:
# Print the values for your selected data for categorization

print(score['children'])
print(score['family_status'])
print(score['purpose'])


0        1
1        1
2        0
3        3
4        0
        ..
21448    1
21449    0
21450    1
21451    3
21452    2
Name: children, Length: 21453, dtype: int64
0                  married
1                  married
2                  married
3                  married
4        civil partnership
               ...        
21448    civil partnership
21449              married
21450    civil partnership
21451              married
21452              married
Name: family_status, Length: 21453, dtype: object
0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21448       housing transactions
21449          purchase of a car
21450                   property
21451          buying my own car
21452               to buy a car
Name: purpose, Length: 21453, dtype: object


In [67]:
# Check the unique values

print(score['children'].unique())
print(score['family_status'].unique())
print(score['purpose'].unique())

[1 0 3 2 4 5]
['married' 'civil partnership' 'widowed' 'divorced' 'never married']
['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']




For 'children', since our hypothesis is to test whether having children affect repaying loan on time. This can be categorized into two groups, 0 = no children and 1, 2, 3, 4, and 5 = have children.

For 'Family_status', since our hypothesis is to test whether family_status affect repaying loan on time. It is possible to categroize into two groups again, married and else. 

For 'purpose', since there are many different reason, I would categorize it based on the key terms, such as 'house', 'car', 'education' and 'wedding' to gain a better understanding of the correlation between purpose and default rate. 

For 'total_income', I will be categorize it into five major groups, 'low-income', 'low-medium income', 'medium-income', 'medium-high income', and finally 'high-income'. This grouping is similar to how income-level are determined in general. 




In [68]:
# Let's write a function to categorize the data based on common topics - children

def number_children(children):
    if children == 0:
        return 'no children'
    
    if children >= 1:
        return 'have children'

print (number_children(4))

have children


In [69]:
# Let's write a function to categorize the data based on common topics - family status

def marriage(status):
    if status == 'married':
        return 'married'
    else:
        return 'not-married'
print(marriage('widowed'))

not-married


In [70]:
# Let's write a function to categorize the data based on common topics - purpose

def purpose(reason):
    if 'house' in reason or 'estate' in reason or 'property' in reason or 'housing' in reason:
        return 'housing'
    elif 'wedding' in reason:
        return 'wedding'
    elif 'car' in reason or 'cars' in reason:
        return 'car'
    elif 'education' in reason or 'university' in reason:
        return 'education'
    
print(purpose('getting higher education'))

education


In [71]:
# Create a column with the categories and count the values for them

score['children_status'] = score['children'].apply(number_children)
score['marriage_status'] = score['family_status'].apply(marriage)
score['purpose_of_loan'] = score['purpose'].apply(purpose)

In [72]:
# Looking through all the numerical data in your selected column for categorization
print(score['total_income'].unique())

[40620.102 17932.802 23341.752 ... 14347.61  39054.888 13127.587]


In [73]:
# Getting summary statistics for the column

print(score['children_status'].value_counts())
print(score['marriage_status'].value_counts())
print(score['purpose_of_loan'].value_counts())

no children      14090
have children     7363
Name: children_status, dtype: int64
married        12339
not-married     9114
Name: marriage_status, dtype: int64
housing      10810
car           4306
education     3605
wedding       2324
Name: purpose_of_loan, dtype: int64


As mentioned before: for 'total_income', I will be categorize it into five major groups, 'low-income', 'low-medium income', 'medium-income', 'medium-high income', and finally 'high-income'. This grouping is similar to how income-level are determined in general and will provide a better representation of the group. 


In [74]:
# Let's write a function to categorize numerical column - total income

def income_range(income):
    if income < 25000:
        return 'low-income'
    if 25000 <= income <= 49999:
        return 'low-medium income'
    if 50000 <= income <= 99999:
        return 'medium income'
    if 100000 <= income <= 249999:
        return 'medium-high income'
    if income >= 250000:
        return 'high-income'
    
print(income_range(310231))

high-income


In [75]:
# Creating column with categories

score['income_status'] = score['total_income'].apply(income_range)


In [76]:
# Count each categories values to see the distribution

print(score['income_status'].value_counts())

low-income            12851
low-medium income      7282
medium income          1221
medium-high income       93
high-income               6
Name: income_status, dtype: int64


Now that several columns are grouped based *my* logical reasoning, we will be testing the hypothese in the next steps.

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [77]:
# Check the children data and paying back on time

print(score.groupby('children_status')['debt'].value_counts())

# Total number of default in Children_status

total_debt_c = score['children_status'][score['debt'] == 1].count()

# Default rate by category

print(score.groupby('children_status')['debt'].sum() / score.groupby('children_status')['debt'].size() *100)

# True default rate

print(score.groupby('children_status')['debt'].sum() / total_debt_c *100)


children_status  debt
have children    0        6685
                 1         678
no children      0       13027
                 1        1063
Name: debt, dtype: int64
children_status
have children    9.208203
no children      7.544358
Name: debt, dtype: float64
children_status
have children    38.943136
no children      61.056864
Name: debt, dtype: float64


**Conclusion**

Default rate by category - which takes the number of defaults within each category and divided by the total of that category. Reflecting the percentage of default appearance within that specific category.

True default rate - taking the number of defaults within each category and divided by the total defaults of *all* categories under this column. Reflecting the percentage of default appearance based on the category.

Although we will need statistical analyses to determine whether the identified differences are significant, comments will be made towards the overall appearance of default based on this category. 

The difference between clients with children and with no children for default rate by category is subtle, suggesting that no group is more likely than other to have default within this specific category. 

However, the true default rate differs drastically between clients with children and no children. 23% more of client with no children have a default, making up 61% of the population that has default. 

Based on the data, we can conclude that there is a correlation between having children and pay back on time, although no group is more likely to have a default than others but clients with no children made up 61% of the population that has default. While clients with children made up 38%.


**Is there a correlation between family status and paying back on time?**

In [78]:
# Check the family status data and paying back on time

print(score.groupby('marriage_status')['debt'].value_counts())

# Total number of default in marriage_status

total_debt_m = score['marriage_status'][score['debt'] == 1].count()

# Default rate by category - marriage

print(score.groupby('marriage_status')['debt'].sum() / score.groupby('marriage_status')['debt'].size() *100)

# True default rate

print(score.groupby('marriage_status')['debt'].sum() / total_debt_m *100)



marriage_status  debt
married          0       11408
                 1         931
not-married      0        8304
                 1         810
Name: debt, dtype: int64
marriage_status
married        7.545182
not-married    8.887426
Name: debt, dtype: float64
marriage_status
married        53.475014
not-married    46.524986
Name: debt, dtype: float64


**Conclusion**

Default rate by category - which takes the number of defaults within each category and divided by the total of that category. Reflecting the percentage of default appearance within that specific category.

True default rate - taking the number of defaults within each category and divided by the total defaults of *all* categories under this column. Reflecting the percentage of default appearance based on the category.

Although we will need statistical analyses to determine whether the identified differences are significant, comments will be made towards the overall appearance of default based on this category. 

The difference between clients that are married or not-married for default rate by category is subtle, suggesting that no group is more likely than other to have default within this specific category. 

However, the true default rate differs a bit more between married and not-married clients. married clients contribute ~7% more to the default population, making up 53% of the population that has default.

Based on the data, we can conclude that there is a correlation (appeared smaller than children) between marraige_status and pay back on time, although no group is more likely to have a default than others but married clients made up 53% of the population that has default, while not-married clients at 46%. 

**Is there a correlation between income level and paying back on time?**

In [79]:
# Check the income level data and paying back on time

print(score.groupby('income_status')['debt'].value_counts())

# Total number of default in income_status

total_debt_i = score['income_status'][score['debt'] == 1].count()

# Default rate by category - income

print(score.groupby('income_status')['debt'].sum() / score.groupby('income_status')['debt'].size() *100)

# True default rate

print(score.groupby('income_status')['debt'].sum() / total_debt_i *100)





income_status       debt
high-income         0           5
                    1           1
low-income          0       11785
                    1        1066
low-medium income   0        6699
                    1         583
medium income       0        1135
                    1          86
medium-high income  0          88
                    1           5
Name: debt, dtype: int64
income_status
high-income           16.666667
low-income             8.295074
low-medium income      8.006042
medium income          7.043407
medium-high income     5.376344
Name: debt, dtype: float64
income_status
high-income            0.057438
low-income            61.229179
low-medium income     33.486502
medium income          4.939690
medium-high income     0.287191
Name: debt, dtype: float64


**Conclusion**

Default rate by category - which takes the number of defaults within each category and divided by the total of that category. Reflecting the percentage of default appearance within that specific category.

True default rate - taking the number of defaults within each category and divided by the total defaults of *all* categories under this column. Reflecting the percentage of default appearance based on the category.

Although we will need statistical analyses to determine whether the identified differences are significant, comments will be made towards the overall appearance of default based on this category. 

The difference between different income level for default rate by category is subtle except for high-income due to small sample sizes, suggesting that no group is more likely than other to have default within this specific category. 

However, the true default rate differs a bit more between different income levels. low-income and low-medium income made up ~94% of the total population with default. While three other levels contribute insignificantly to the total population.

Based on the data, we can conclude that there is a correlation between income level and pay back on time, although no group is more likely to have a default than others but low-income and low-medium income clients made up ~94% of the population that has default, while others at 6%. 

**How does credit purpose affect the default rate?**

In [80]:
# Check the percentages for default rate for each credit purpose 

print(score.groupby('purpose_of_loan')['debt'].value_counts())

# Total number of default in purpose_of_loan

total_debt_r = score['purpose_of_loan'][score['debt'] == 1].count()

# Default rate by category - purpose

print(score.groupby('purpose_of_loan')['debt'].sum() / score.groupby('purpose_of_loan')['debt'].size() *100)

# True default rate

print(score.groupby('purpose_of_loan')['debt'].sum() / total_debt_r *100)


purpose_of_loan  debt
car              0        3903
                 1         403
education        0        3274
                 1         331
housing          0       10028
                 1         782
wedding          0        2138
                 1         186
Name: debt, dtype: int64
purpose_of_loan
car          9.359034
education    9.181692
housing      7.234043
wedding      8.003442
Name: debt, dtype: float64
purpose_of_loan
car          23.678026
education    19.447709
housing      45.945946
wedding      10.928320
Name: debt, dtype: float64


In [86]:
default_pivot = score.pivot_table(
index = ['purpose_of_loan', 'children_status', 'income_status','marriage_status'],
values = 'debt',
aggfunc = 'mean'
)

print(default_pivot)

                                                                        debt
purpose_of_loan children_status income_status      marriage_status          
car             have children   low-income         married          0.107492
                                                   not-married      0.127193
                                low-medium income  married          0.096354
                                                   not-married      0.149606
                                medium income      married          0.086420
                                                   not-married      0.125000
                                medium-high income married          0.000000
                                                   not-married      0.000000
                no children     low-income         married          0.073171
                                                   not-married      0.103586
                                low-medium income  married          0.065026

**Conclusion**

Default rate by category - which takes the number of defaults within each category and divided by the total of that category. Reflecting the percentage of default appearance within that specific category.

True default rate - taking the number of defaults within each category and divided by the total defaults of *all* categories under this column. Reflecting the percentage of default appearance based on the category.

Although we will need statistical analyses to determine whether the identified differences are significant, comments will be made towards the overall appearance of default based on this category. 

The difference between different purpose for default rate by category is subtle, suggesting that no group is more likely than other to have default within this specific category. 

However, the true default rate differs a bit more between different the purpose of loan. housing made up about ~46% of the total population with default. Following by car at 23%, education at 19% and finally wedding at 10%.

Based on the data, we can conclude that there is a correlation between purpose of loan and pay back on time, although no group is more likely to have a default than others but for house and car made up ~70% of the population that has default, while others at 30%. 


# General Conclusion 

Lastly, the dataset has been:

1. preprocessed by fixing the 'education' column, where strings are now identical by making them lowercase with str.lower(). 

The problematic entries -1 and 20 found in 'children' has been replaced with logical values. 

'days_employed' consisted of ~73% of negative values, which was problematic. An absoulte has been taken for the whole column, changing negative numbers to positive. 

The 0 found in 'dob_years' column has been replaced with a logical value of 30.

Some registers under 'family_status' have been modified to make it more clear. 

One entry of 'XNA' found in 'gender' column has been removed.

'entrepreneur' was merged with 'business' for better representaion in 'income_type' column. While leaving 'student', 'paternity/maternity leave' untouched as unsure of where they qualify. 

71 obvious duplicates were found using .duplicated() and removed with drop_duplicates().

Newly processed dataset now has 21453 rows, as compared to 21525 to begin with. So a total of 72 rows were removed, counting for roughly 0.3% of the dataset. 

2. Missing values in 'days_employed' and 'total_income' have been filled with median() due to the presence of outliers. 

3. Three categorical column and one numerical were chosen to categorize. 

Recall the hypothesis:

    1. There is an association between having kids and repaying a loan on time. ('children', 'debt') 
    2. There is an association between marital status and repaying a loan on time. ('family_status','debt')
    3. There is an association between income level and repaying a loan on time. ('total_income','debt')
    4. Different loan purposes affect on-time loan repayment ('purpose', 'debt')

'children','family_status', and 'purpose' are categorical columns, while 'total_income' is numerical. Currently, there are multiple entries under each column, to get a better idea, they should be categorized into a clear, more representative way.



In general, there appeared to be an association between purpose of loan, income-level, having children, and marriage with defaults. Although no statistical analyses were performed to determine whether the identified differences are significant. We can conclude that:

In a pool of people that have defaults, the characteristics are:

The purpose of loan is likely to be for housing or car;
or
They do not have children;
or
They are married;
or
They have low- or low-medium income. 

From those charactersitics, income-level (90%) and purpose of loan (70%) appeared to be more representative even with more categories. 

